In [1]:
# 네트워크 테스트하는 함수 구현하기

# MNIST 데이터셋 불러오는 부분에서 train = True -> False로 변환
'''
dataset = datasets.MNIST(download=True, root='./', train=False, transform = transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
'''

# 트레이닝 시작 부분에서 아래 코드 삭제 후 
'''
for epoch in range(1, num_epoch + 1) :
    net.train()
'''

'''
with torch.no_grad():
    net.eval()
'''

'\nwith torch.no_grad():\n    net.eval()\n'

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [3]:
# train에 필요한 parameter 설정하기

lr = 1e-3
batch_size = 64
num_epoch = 10

# 학습된 data가 저장될 dir
ckpt_dir = './checkpoint'
# tensorboard가 저장될 dir
log_dir = './log'
# device 정보 입력
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# classifier network 구축

class Net(nn.Module) :
    # Layer 초기화 단계
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)
        
        
    # Layer 구축 단계
    def forward(self,x) :
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)
    
        
        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)
        
        return x

In [5]:
# 네트워크를 저장하거나 불러오는 함수 (save(), load())

def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

In [6]:
# MNIST 데이터 셋 불러오기

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=False, transform = transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

In [7]:
# 네트워크 설정 및 손실함수 설정하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

net, optim = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

In [8]:
# 네트워크 학습을 위한 for문 구현 (트레이닝 시작)

with torch.no_grad():
    net.eval()
    
    loss_arr = []
    acc_arr = []
    
    for batch, (input, label) in enumerate(loader, 1) :
        input = input.to(device)
        label = label.to(device)
        
        output = net(input)
        pred = fn_pred(output)
        
        #optim.zero_grad()
        
        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)
        
        #loss.backward()
        
        #optim.step()
        
        loss_arr += [loss.item()]
        acc_arr += [acc.item()]
        
        print('TEST: BATCH %04d/%04d | LOSS %.4f | ACC %.4f' % 
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    #writer.add_scalar('loss', np.mean(loss_arr),epoch)
    #writer.add_scalar('loss', np.mean(acc_arr),epoch)
    
    #save(ckpt_dir = ckpt_dir, net =net, optim = optim, epoch = epoch)

#writer.close()

/home/blue/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


TEST: BATCH 0001/0157 | LOSS 0.0210 | ACC 1.0000
TEST: BATCH 0002/0157 | LOSS 0.0169 | ACC 1.0000
TEST: BATCH 0003/0157 | LOSS 0.0153 | ACC 1.0000
TEST: BATCH 0004/0157 | LOSS 0.0209 | ACC 0.9961
TEST: BATCH 0005/0157 | LOSS 0.0257 | ACC 0.9938
TEST: BATCH 0006/0157 | LOSS 0.0286 | ACC 0.9922
TEST: BATCH 0007/0157 | LOSS 0.0339 | ACC 0.9911
TEST: BATCH 0008/0157 | LOSS 0.0363 | ACC 0.9902
TEST: BATCH 0009/0157 | LOSS 0.0335 | ACC 0.9913
TEST: BATCH 0010/0157 | LOSS 0.0327 | ACC 0.9922
TEST: BATCH 0011/0157 | LOSS 0.0322 | ACC 0.9929
TEST: BATCH 0012/0157 | LOSS 0.0359 | ACC 0.9909
TEST: BATCH 0013/0157 | LOSS 0.0352 | ACC 0.9904
TEST: BATCH 0014/0157 | LOSS 0.0337 | ACC 0.9911
TEST: BATCH 0015/0157 | LOSS 0.0363 | ACC 0.9896
TEST: BATCH 0016/0157 | LOSS 0.0403 | ACC 0.9883
TEST: BATCH 0017/0157 | LOSS 0.0413 | ACC 0.9881
TEST: BATCH 0018/0157 | LOSS 0.0402 | ACC 0.9887
TEST: BATCH 0019/0157 | LOSS 0.0393 | ACC 0.9893
TEST: BATCH 0020/0157 | LOSS 0.0449 | ACC 0.9875
TEST: BATCH 0021/015

In [9]:
%load_ext tensorboard

In [10]:
tensorboard --logdir='~/Desktop/Daeheon/conda/Deeplearning/MNIST/log'